In [9]:
# UNIX
! rm net_names.txt
! ls database >> net_names.txt

In [10]:
# NOTES
# The input file 'Fanconi anemia and checkpoint recovery.txt'
# was fixed manually, because the node 'DSB' is not formatted in a uniform way.

In [11]:
file = open('net_names.txt')
net_names = [name.rstrip() for name in file]

In [24]:
network = 'database/'+net_names[0]

file = open(network)
file_content = [line.rstrip() for line in file]

In [25]:
clean_file = [line for line in file_content if line[0]=='#']
node_names = clean_file[0][3:].split(' ')
input_nodes = [line[2:] for line in clean_file[1:]]
n = len(node_names)

n

67

In [26]:
node_names

['AKT',
 'Apo',
 'ATM',
 'BAD',
 'BAX',
 'BCL2',
 'Bclxl',
 'BID',
 'Cas3',
 'Cas8',
 'Cas9',
 'CDK2',
 'CDK4',
 'CDK6',
 'CIAP',
 'cJun',
 'Cl_PARP',
 'CYCD',
 'CYCE',
 'Cytc',
 'DNAdam',
 'DNAPK',
 'ERK',
 'Fas',
 'FasL',
 'FLIP',
 'gp130',
 'GROWTH',
 'IKK',
 'IL6',
 'JAK1',
 'JAK2',
 'JNK',
 'MAPK',
 'MDM',
 'MEK1',
 'MEKK',
 'MEKK2',
 'MITO',
 'MKK4',
 'MYC',
 'NIK',
 'p21',
 'p27',
 'p53',
 'PI3K',
 'pIKB',
 'PIP3',
 'pNFKB',
 'pRB',
 'Prot',
 'pSTAT3',
 'PTEN',
 'RAC',
 'RAF',
 'RAS',
 'RIP',
 'Smac',
 'STAT3',
 'STRESS',
 'TRAF3',
 'XIAP',
 'X',
 'SHP1',
 'TNFA',
 'Bort',
 'TNFAR']

In [27]:
sorted_node_names = list(sorted(node_names, key = len, reverse = True))

We are sorting the node names according to their length, and in reverse order, 
because some names might be included in other names (e.g. IL2 in IL21).
By replacing IL2 with a number, e.g. 5,  we would turn IL21 into 21

In [28]:
input_nodes

['AKT (PIP3, AKT)',
 'Apo (Cl_PARP, Apo)',
 'ATM (Cas3, DNAdam, ATM)',
 'BAD (AKT, BAD)',
 'BAX (BCL2, BID, Bclxl, p53, BAX)',
 'BCL2 (BAD, p53, pNFKB, pSTAT3, BCL2)',
 'Bclxl (BAD, BAX, p53, pNFKB, pSTAT3, Bclxl)',
 'BID (BCL2, Bclxl, Cas8, Fas, STRESS, BID)',
 'Cas3 (Cas8, Cas9, XIAP, Cas3)',
 'Cas8 (FLIP, Fas, FasL, Cas8)',
 'Cas9 (AKT, Cytc, XIAP, Cas9)',
 'CDK2 (CYCE, p21, CDK2)',
 'CDK4 (CYCD, p21, p27, CDK4)',
 'CDK6 (CYCD, CDK6)',
 'CIAP (TNFAR, CIAP)',
 'cJun (JNK, cJun)',
 'Cl_PARP (Cas3, Cl_PARP)',
 'CYCD (AKT, ERK, MYC, CYCD)',
 'CYCE (MYC, CYCE)',
 'Cytc (MITO, Cytc)',
 'DNAdam (Cas3, STRESS, DNAdam)',
 'DNAPK (ATM, DNAPK)',
 'ERK (MAPK, ERK)',
 'Fas (FasL, p53, Fas)',
 'FasL (Fas, cJun, FasL)',
 'FLIP (pNFKB, FLIP)',
 'gp130 (Cas3, gp130)',
 'GROWTH (pRB, GROWTH)',
 'IKK (AKT, NIK, RIP, IKK)',
 'IL6 (pNFKB, IL6)',
 'JAK1 (IL6, SHP1, gp130, JAK1)',
 'JAK2 (IL6, SHP1, gp130, JAK2)',
 'JNK (MKK4, Prot, JNK)',
 'MAPK (MEK1, MAPK)',
 'MDM (AKT, ATM, p53, MDM)',
 'MEK1 (RAF, ME

In [29]:
input_nodes_labels = []

for line in input_nodes:
    for i in range(n):
        line = line.replace(sorted_node_names[i],str(i))
    input_nodes_labels.append(line)

In [30]:
input_nodes_labels

['44 (36, 44)',
 '45 (0, 45)',
 '46 (14, 1, 46)',
 '47 (44, 47)',
 '48 (13, 49, 5, 60, 48)',
 '13 (47, 60, 9, 3, 13)',
 '5 (47, 48, 60, 9, 3, 5)',
 '49 (13, 5, 15, 51, 4, 49)',
 '14 (15, 16, 40, 14)',
 '15 (26, 51, 25, 15)',
 '16 (44, 24, 40, 16)',
 '17 (23, 58, 17)',
 '18 (22, 58, 59, 18)',
 '19 (22, 19)',
 '20 (12, 20)',
 '21 (54, 21)',
 '0 (14, 0)',
 '22 (44, 50, 56, 22)',
 '23 (56, 23)',
 '24 (32, 24)',
 '1 (14, 4, 1)',
 '6 (46, 6)',
 '50 (29, 50)',
 '51 (25, 60, 51)',
 '25 (51, 21, 25)',
 '26 (9, 26)',
 '7 (14, 7)',
 '2 (61, 2)',
 '52 (44, 57, 65, 52)',
 '53 (9, 53)',
 '27 (53, 41, 7, 27)',
 '28 (53, 41, 7, 28)',
 '54 (33, 37, 54)',
 '29 (30, 29)',
 '55 (44, 46, 60, 55)',
 '30 (63, 30)',
 '31 (30, 31)',
 '8 (62, 8)',
 '32 (48, 32)',
 '33 (8, 33)',
 '56 (50, 31, 3, 56)',
 '57 (11, 57)',
 '58 (44, 18, 55, 56, 60, 58)',
 '59 (44, 18, 56, 60, 59)',
 '60 (6, 54, 55, 37, 60)',
 '34 (53, 7, 34)',
 '35 (52, 35)',
 '36 (36, 38, 36)',
 '9 (37, 66, 35, 3, 9)',
 '61 (17, 18, 19, 61)',
 '37 (4

In [31]:
edge_list = []

for line in input_nodes_labels:
    formatted_line =  line.replace(' ','').replace(')','').split('(')
    output = formatted_line[0]
    inputs = formatted_line[1].split(',')
    for i in inputs:
        if i != output: # I'm removing self-loops, as the motif code doesn't account for them
            edge_list.append([i,output,'directed',1])
#print(edge_list)

In [32]:
import networkx as nx
import pandas as pd
import pyvis
from pyvis.network import Network

# https://towardsdatascience.com/visualizing-networks-in-python-d70f4cbeb259

In [33]:
df = pd.DataFrame(edge_list,columns=['Source','Target','Type','weight'])
df[:3]

,Source,Target,Type,weight
0,36,44,directed,1
1,0,45,directed,1
2,14,46,directed,1


In [34]:
G = nx.from_pandas_edgelist(df,source='Source',target='Target',edge_attr = 'weight')
net = Network(notebook = True, directed = True)
net.from_nx(G)

In [35]:
net.show("example.html")

In [ ]:
# I keep finding zero motifs with Milo's code.
# I need to make Shade's code run, and see if the result changes.